## 📚 Prerequisites

Before running this notebook, ensure you have configured Azure AI services, set the appropriate configuration parameters, and set up a Conda environment to ensure reproducibility. You can find the setup instructions and how to create a Conda environment in the [REQUIREMENTS.md](REQUIREMENTS.md) file.


In [1]:
import os

# Define the target directory (change yours)
target_directory = r"C:\Users\pablosal\Desktop\gbbai-chat-with-your-database"

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-chat-with-your-database


In [2]:
from src.database.azuresql import AzureSQLManager

DATABASE = "dev-sql-server"
az_sql_client = AzureSQLManager(DATABASE)

In [3]:
from src.aoai.azure_openai import AzureOpenAIManager

az_client = AzureOpenAIManager()

## Knowledge Engineering

In this section, we will discuss the process of knowledge engineering in the context of our project. This involves extracting tables from databases, converting them into natural language, and vectorizing the content. The vectorization process enhances the relevance of our search system by enabling semantic search capabilities. This way, we can create rules and knowledge bases for our artificial intelligence systems to provide more accurate and contextually relevant search results.

In [4]:
tables_names = az_sql_client.get_table_names()
schema = az_sql_client.process_schema(tables_names[0])

In [5]:
system_message = """You are an AI assistant that helps people find information.  Act as a Senior Database Engineer, capable of understanding complex 
relationships between fields and tables. Your task involves interpreting and analyzing 
the schema of SQL database tables provided in JSON format. You examine the structure,
understand the purpose of each field, and identify the interconnections between fields. 
Your objective is to produce a comprehensive summary that elucidates the table's design
and its potential applications. This summary should highlight the table's intended use, 
emphasizing the key data it contains. Your expertise is essential for demystifying the 
navigation and optimization of complex database systems, facilitating users to comprehend
the significance and operational context of their data assets efficiently. Take your time."""

In [6]:
query = f"""
Please examine the schema of the table thoroughly. Provide an in-depth analysis that not only explains the design and structure of the table but also outlines its potential applications and the types of queries it can support.
Also provide atleast 10 queries in natural lengauge that can be run on the table.
Please take your time.

Here is the schema of the table: {schema}
"""

In [7]:
content = az_client.generate_chat_response(
    conversation_history=[],
    query=query,
    system_message_content=system_message,
    max_tokens=1000,
)

2024-02-07 01:17:55,106 - micro - MainProcess - INFO     Sending request to OpenAI with query: 
Please examine the schema of the table thoroughly. Provide an in-depth analysis that not only explains the design and structure of the table but also outlines its potential applications and the types of queries it can support.
Also provide atleast 10 queries in natural lengauge that can be run on the table.
Please take your time.

Here is the schema of the table: {'detroit_tigers_baseball_stats': [{'column_number': 0, 'column_name': 'name', 'data_type': 'nvarchar'}, {'column_number': 1, 'column_name': 'position', 'data_type': 'nvarchar'}, {'column_number': 2, 'column_name': 'Games_Played', 'data_type': 'tinyint'}, {'column_number': 3, 'column_name': 'At_Bats', 'data_type': 'smallint'}, {'column_number': 4, 'column_name': 'Hits', 'data_type': 'tinyint'}, {'column_number': 5, 'column_name': 'Home_Runs', 'data_type': 'tinyint'}, {'column_number': 6, 'column_name': 'RBIs', 'data_type': 'tinyint'

In [8]:
embedding = az_client.generate_embedding(input_text=content)

## Indexing

In [9]:
import os
import random
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [22]:
# Set the service endpoint and API key from the environment
# Create an SDK client
endpoint = os.environ["AZURE_AI_SEARCH_SERVICE_ENDPOINT"]
search_client = SearchClient(
    endpoint=endpoint,
    index_name="query-dev-index",
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]),
)

In [14]:
tables_names = az_sql_client.get_table_names()
schema = az_sql_client.process_schema(tables_names[0])

In [28]:
chunked_content_docs = []

In [29]:
import random

# Iterate over each table name
for table in tables_names:
    # Process the schema of the table
    schema = az_sql_client.process_schema(table)

    # Generate a chat response
    content = az_client.generate_chat_response(
        conversation_history=[],
        query=query,
        system_message_content=system_message,
        max_tokens=1000,
    )
    # Generate a random document ID
    doc_id = str(random.randint(1, 1000000))

    # Generate an embedding for the content
    embedding = az_client.generate_embedding(input_text=content)

    # Create a JSON object with the table content, vector, document ID, and table name
    json_data = {
        "table_content": content,
        "table_vector": embedding,
        "document_id": doc_id,
        "table_name": table,
    }

    # Append the JSON object to the list of documents
    chunked_content_docs.append(json_data)

2024-02-07 01:31:54,059 - micro - MainProcess - INFO     Sending request to OpenAI with query: 
Please examine the schema of the table thoroughly. Provide an in-depth analysis that not only explains the design and structure of the table but also outlines its potential applications and the types of queries it can support.
Also provide atleast 10 queries in natural lengauge that can be run on the table.
Please take your time.

Here is the schema of the table: {'detroit_tigers_baseball_stats': [{'column_number': 0, 'column_name': 'name', 'data_type': 'nvarchar'}, {'column_number': 1, 'column_name': 'position', 'data_type': 'nvarchar'}, {'column_number': 2, 'column_name': 'Games_Played', 'data_type': 'tinyint'}, {'column_number': 3, 'column_name': 'At_Bats', 'data_type': 'smallint'}, {'column_number': 4, 'column_name': 'Hits', 'data_type': 'tinyint'}, {'column_number': 5, 'column_name': 'Home_Runs', 'data_type': 'tinyint'}, {'column_number': 6, 'column_name': 'RBIs', 'data_type': 'tinyint'

2024-02-07 01:32:28,595 - micro - MainProcess - INFO     Received response from OpenAI: The provided schema is for a table named 'detroit_tigers_baseball_stats', which likely stores statistical data related to the players of the Detroit Tigers, a professional baseball team. The table has eight columns, each representing different characteristics or statistics related to a player.

1. 'name' (nvarchar): This column is used to store the name of the player. As the data type is 'nvarchar', it can contain both text and numbers, allowing for diverse names.

2. 'position' (nvarchar): This field stores the position of a player on the field. The 'nvarchar' data type indicates that this field can also contain alphanumeric characters, allowing for various positions like 'SS' (Short Stop), 'CF' (Center Field), etc.

3. 'Games_Played' (tinyint): This field records the number of games a player has played. The 'tinyint' data type indicates that this number is likely to be relatively small, generally 

In [32]:
print(f"Uploading batch of {len(chunked_content_docs)} documents...")
result = search_client.upload_documents(documents=chunked_content_docs)
for res in result:
    print("Upload of new document succeeded: {}".format(res.succeeded))

Uploading batch of 2 documents...
Upload of new document succeeded: True
Upload of new document succeeded: True
